In [ ]:
import kagglehub

path = kagglehub.dataset_download("rihabkaci99/fatigue-dataset")

print("Path to dataset files:", path)

In [ ]:
import os

for root, dirs, files in os.walk(path):
    print(root, len(files))


In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
fatigue_shw = path + '/Data/Fatigue'

files = os.listdir(fatigue_shw)[:10]

plt.figure(figsize=(12, 12))
for i, img_name in enumerate(files):
  img = Image.open(os.path.join(fatigue_shw, img_name))
  plt.subplot(2
              , 5, i+1)
  plt.imshow(img)
  plt.axis('off')
  plt.title(img_name)
plt.show()


In [ ]:
non_fatigue = path + '/Data/NonFatigue'
files2 = os.listdir(non_fatigue)[:10]
plt.figure(figsize=(12, 12))
for i, img_name in enumerate(files2):
  img = Image.open(os.path.join(non_fatigue, img_name))
  plt.subplot(2, 5, i+1)
  plt.imshow(img)
  plt.axis('off')
  plt.title(img_name)
plt.show()

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split, Subset

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [ ]:
data = datasets.ImageFolder(root = path + '/Data')
train_set = datasets.ImageFolder(root = path + '/Data', transform = train_transform)
test_set = datasets.ImageFolder(root = path + '/Data', transform = test_transform)

In [ ]:
train = int(0.8* len(data))
test = len(data)- train
train_indices, test_indices = random_split(range(len(data)), [train, test])

In [ ]:
train_data = Subset(train_set, train_indices.indices)
test_data = Subset(test_set, test_indices.indices)



In [ ]:
train_loader = DataLoader(train_data, batch_size = 64, shuffle = True)
test_loader = DataLoader(test_data, batch_size = 64, shuffle = False)

In [ ]:
pip install optuna

In [ ]:
import torch.nn as nn
class CNN(nn.Module):
  def __init__(self, input_channel, output_channel, classes):
    super(CNN, self).__init__()
    layers=[]
    prev_ch = input_channel
    for out_ch in output_channel:
      layers.append(nn.Conv2d(prev_ch, out_ch, kernel_size=3, stride=1, padding=1))
      layers.append(nn.ReLU())
      layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
      prev_ch = out_ch
      self.features = nn.Sequential(*layers)

      self.pool = nn.AdaptiveAvgPool2d((1, 1))

      self.classifier = nn.Sequential(
          nn.Flatten(),
          nn.Linear(prev_ch*1*1, 512),
          nn.ReLU(),
          nn.Dropout(0.5),
          nn.Linear(512, 128),
          nn.ReLU(),
          nn.Dropout(0.5),
          nn.Linear(128, classes)
      )

  def forward(self, x):
    x = self.features(x)
    x = self.pool(x)
    x = self.classifier(x)
    return x


In [ ]:
import optuna
import torch
import torch.nn as nn
 # You can adjust the number of epochs for hyperparameter tuning


In [ ]:
def train(model, train_loader, criterion, optimizer):
  model.train()
  running_loss = 0.0
  correct = 0
  total = 0
  for batch, (images, labels) in enumerate(train_loader):

    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    _, predicted = outputs.max(1)
    total += labels.size(0)

    correct += predicted.eq(labels).sum().item()
  train_loss = running_loss / len(train_loader)
  train_acc = 100. * correct / total
  if batch % 100 == 0:
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
  return train_loss, train_acc



In [ ]:
def test(model, test_loader, criterion):
  model.eval()
  running_loss = 0.0
  correct = 0
  total = 0
  with torch.no_grad():
    for images, labels in test_loader:

      outputs = model(images)
      loss = criterion(outputs, labels)
      running_loss += loss.item()
      _, predicted = outputs.max(1)
      total += labels.size(0)
      correct += predicted.eq(labels).sum().item()
  test_loss = running_loss / len(test_loader)
  test_acc = 100. * correct / total
  print(f'Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.2f}%')
  return test_loss, test_acc

In [ ]:
def objective(trial):
  n_conv = trial.suggest_int("n_conv", 1, 4)

  base_ch = trial.suggest_categorical("base_ch", [16, 32, 48, 64])

  channels_list = [int(base_ch * (2 ** i)) for i in range(n_conv)]
  lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)

  model = CNN(input_channel=3, output_channel=channels_list, classes=len(data.classes))


  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)

  num_epochs = 5
  for epoch in range(num_epochs):
      train(model, train_loader, criterion, optimizer)


  test_loss, accuracy = test(model, test_loader, criterion)
  return accuracy

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5, timeout=None)

In [ ]:
study = study.best_params

In [ ]:
study

In [ ]:
n_conv_best = study['n_conv']
base_ch_best = study['base_ch']

channels_list_best = [int(base_ch_best * (2 ** i)) for i in range(n_conv_best)]

model = CNN(
    input_channel=3,
    output_channel=channels_list_best,
    classes=len(data.classes)
)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=study['lr'])

epochs = 20
for epoch in range(epochs):
  train(model=model, train_loader=train_loader, criterion=criterion, optimizer=optimizer)
  test(model=model, test_loader=test_loader, criterion=criterion)

In [ ]:
torch.save(model.state_dict(), "best_cnn_model.pt")


In [ ]:

all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for images, labels in test_loader:

        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
metrics=[]
def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    metrics.append([accuracy, precision, recall, f1])
    metrics_df = pd.DataFrame(metrics, columns=['Accuracy', 'Precision', 'Recall', 'F1 Score'])
    return metrics


In [ ]:
calculate_metrics(all_labels, all_preds)